<a href="https://colab.research.google.com/github/owaiskhan2501000/Assignment-005/blob/main/4k_Video_Upscaler_Colab_(Real_ESRGAN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4k Video Upscaler Colab (Real-ESRGAN)

Adapted from: [Real-ESRGAN](https://github.com/xinntao/Real-ESRGAN)

Made with ❤️ by: [yuvraj108c](https://github.com/yuvraj108c)

Github repository: https://github.com/yuvraj108c/4k-video-upscaler-colab

# 1. Setup (~1 minute)

In [3]:
import torch
from PIL import Image
import cv2, os, subprocess
from google.colab import drive
from tqdm import tqdm

# Check for GPU availability
assert torch.cuda.is_available(), "GPU not detected.. Please change runtime to GPU"

# Clone the Real-ESRGAN repository
!git clone -q https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN

# Install PyTorch (Colab usually has compatible versions pre-installed)
!pip install -q torch torchvision torchaudio

# Install required dependencies
!pip install -q basicsr facexlib gfpgan ffmpeg ffmpeg-python
!pip install -q -r requirements.txt

# Setup the Real-ESRGAN package
!python setup.py develop

# Mount Google Drive if needed
mount_drive = False  # @param {type:"boolean"}

if mount_drive:
    drive.mount('/content/gdrive/')


/content/Real-ESRGAN/Real-ESRGAN
/usr/local/lib/python3.11/dist-packages/setuptools/__init__.py:94: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
running develop
/usr/local/lib/python3.11/dist-packages/setuptools/command/develop.py:41: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://github.com/pypa/setuptools/issu

# 2. Upscale video

- The upscaled video will be saved to `output_dir`
- If google drive is mounted, it will be also saved at `MyDrive/Upscaled Videos (REAL-ESRGAN)`


In [ ]:
import torch
import os
import cv2
import subprocess
from google.colab import drive
from tqdm import tqdm

# Ensure video path exists
video_path = "/content/input.mp4"  # @param {type:"string"}
output_dir = "/content/"  # @param {type:"string"}
resolution = "4k (3840 x 2160)"  # @param ["FHD (1920 x 1080)", "2k (2560 x 1440)", "4k (3840 x 2160)","2 x original", "3 x original", "4 x original"] {type:"string"}
model = "RealESRGAN_x4plus"  # @param ["RealESRGAN_x4plus", "RealESRGAN_x4plus_anime_6B", "realesr-animevideov3"]

assert os.path.exists(video_path), "Error: Video file does not exist!"

# Load video and get dimensions
video_capture = cv2.VideoCapture(video_path)
video_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
video_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
aspect_ratio = video_width / video_height

# Determine final resolution
final_width, final_height = None, None

if resolution == "FHD (1920 x 1080)":
    final_width, final_height = 1920, 1080
elif resolution == "2k (2560 x 1440)":
    final_width, final_height = 2560, 1440
elif resolution == "4k (3840 x 2160)":
    final_width, final_height = 3840, 2160
elif resolution == "2 x original":
    final_width, final_height = 2 * video_width, 2 * video_height
elif resolution == "3 x original":
    final_width, final_height = 3 * video_width, 3 * video_height
elif resolution == "4 x original":
    final_width, final_height = 4 * video_width, 4 * video_height

# Adjust aspect ratio if needed
if "original" not in resolution:
    if aspect_ratio == 1.0:
        final_height = final_width
    elif aspect_ratio < 1.0:
        final_width, final_height = final_height, final_width

# Ensure scale factor is even (for ESRGAN compatibility)
scale_factor = max(final_width / video_width, final_height / video_height)
while int(video_width * scale_factor) % 2 != 0 or int(video_height * scale_factor) % 2 != 0:
    scale_factor += 0.01

print(f"Upscaling from {video_width}x{video_height} to {final_width}x{final_height}, scale_factor={scale_factor}")

# Run Real-ESRGAN
!python inference_realesrgan_video.py -n {model} -i "{video_path}" -o "{output_dir}" --outscale {scale_factor}

# Rename and crop output video
video_name = os.path.splitext(os.path.basename(video_path))[0]
upscaled_video_path = f"{output_dir}{video_name}_out.mp4"
final_video_name = f"{video_name}_upscaled_{final_width}x{final_height}.mp4"
final_video_path = os.path.join(output_dir, final_video_name)

if "original" not in resolution:
    print("Cropping video to fit selected resolution...")
    crop_command = f"""
    ffmpeg -y -loglevel error -hwaccel cuda -i "{upscaled_video_path}" -vf "crop={final_width}:{final_height}" -c:v h264_nvenc "{final_video_path}"
    """
    subprocess.run(crop_command, shell=True)

print(f"Upscaled video saved to: {final_video_path}")

# Mount Google Drive (if required)
mount_drive = False  # @param {type:"boolean"}

if mount_drive:
    drive.mount('/content/gdrive/')
    drive_folder = "/content/gdrive/MyDrive/Upscaled_Videos_REAL_ESRGAN"
    os.makedirs(drive_folder, exist_ok=True)
    drive_save_path = os.path.join(drive_folder, final_video_name)

    # Move the upscaled video to Google Drive
    subprocess.run(f"mv '{final_video_path}' '{drive_save_path}'", shell=True)
    print(f"Saved to Google Drive: {drive_save_path}")

# Remove intermediate output file
!rm "{upscaled_video_path}"


Upscaling from 1920x1080 to 3840x2160, scale_factor=2.0
Downloading: "https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth" to /content/Real-ESRGAN/Real-ESRGAN/weights/RealESRGAN_x4plus.pth

100% 63.9M/63.9M [00:00<00:00, 276MB/s]
inference:   3% 10/300 [03:13<1:33:13, 19.29s/frame]

# 3. Disconnect runtime

In [ ]:
from google.colab import runtime

disconnect_when_finish = False  #@param{type:"boolean"}

if disconnect_when_finish:
  runtime.unassign()